<img heigth="8" src="https://i.imgur.com/BhG5KQ3.png" alt="pbs-enae">

<h1 align="left"><i>Web scraping</i> para el análisis de la competencia empresarial</h1>
<h2 align="left"><i>Introduction to Web Scraping with BeautifulSoup</i></h2>

<p align="left">
  <h3><a href="https://joefaver.dev">Joseph F. Vergel-Becerra</a> | Aplicaciones de Python - Tools and Skill Courses</h3>
  <br>
  <b>Last updated:</b> <i>18/11/2023</i>
  <br><br>
  <a href="#referencias">Referencias</a> •
  <a href="#contribuir">Contribuir</a>
  <br><br>
</p>
<table align="left">
  <td>
      <a href="https://img.shields.io/badge/version-0.1.0-blue.svg?cacheSeconds=2592000">
        <img src="https://img.shields.io/badge/version-0.1.0-blue.svg?cacheSeconds=2592000" alt="Version" height="18">
      </a>
  </td>
  <td>
    <a href="https://colab.research.google.com/github/joefavergel/pbs-enae-python-applications-course/blob/main/4-modern-web-scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
      </a>
  </td>
  <td>
    <a href="https://github.com/joefavergel/pbs-enae-python-applications-course" target="_parent"><img src="https://img.shields.io/github/forks/joefavergel/pbs-enae-python-beginners-course?style=social" alt="Fork"/>
    </a>
  </td>
</table>
<br>
<br>

---

En este tutorial de *web scraping en Python*, aprenderemos cómo extraer información de una página de [IMDb](http://www.imdb.com) utilizando la biblioteca `BeautifulSoup`. El objetivo es raspar datos de películas, procesar el código HTML y transformarlo en datos tabulares para un análisis más detallado.

1. Instalación y configuración: Para comenzar, instale las bibliotecas necesarias, como '`BeautifulSoup`4', 'requests' y 'pandas', utilizando pip o conda.

2. Realizar una solicitud HTTP: Importe la biblioteca 'requests' y envíe una solicitud GET a la URL objetivo de IMDb. Asegúrese de que la solicitud sea exitosa y almacene el contenido en una variable.

3. Crear objeto `BeautifulSoup`: Importe '`BeautifulSoup`' de 'bs4' y cree un objeto con el contenido de la página, utilizando el analizador 'html.parser'.

4. Extracción de datos: Utilice selectores CSS y métodos de `BeautifulSoup`, como 'find' y 'find_all', para localizar y extraer información relevante, como el título, la fecha de lanzamiento, la calificación y otros detalles de las películas.

5. Creación de un DataFrame: Importe 'pandas' y cree un DataFrame vacío con las columnas deseadas. Itere sobre los elementos extraídos y almacene la información en el DataFrame.

6. Limpieza y preprocesamiento: Realice la limpieza de datos necesaria para eliminar caracteres no deseados y formatear adecuadamente los datos, como convertir las calificaciones en números decimales y las fechas en objetos datetime.

7. Análisis y exportación: Ahora que tiene un DataFrame limpio y bien estructurado, puede realizar análisis adicionales, visualizaciones o exportar los datos a un archivo CSV o Excel.

Siguiendo estos pasos, podrá raspar y procesar datos de IMDb utilizando `BeautifulSoup` en Python, y transformarlos en datos tabulares para análisis y uso en proyectos futuros.

## Caso de uso: Facil *web scraping* con `langchain` 

[Investigación web](https://blog.langchain.dev/automating-web-research/) es una de las aplicaciones estrella de los grandes modelos de lenguaje (LLM por sus siglas en ingles):

* Los usuarios han [destacado](https://twitter.com/GregKamradt/status/1679913813297225729?s=20) que es una de sus herramientas de IA más deseadas.
* Repositorios OSS como [gpt-researcher](https://github.com/assafelovic/gpt-researcher) están ganando popularidad.

![Descripción de la imagen](https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/static/img/web_scraping.png)

## Visión general

Recopilar contenido de la web tiene varios componentes:

* `Búsqueda`: Consulta una URL (por ejemplo, utilizando `GoogleSearchAPIWrapper`).
* `Carga`: URL a HTML (por ejemplo, utilizando `AsyncHtmlLoader`, `AsyncChromiumLoader`, etc.).
* `Transformación`: HTML a texto formateado (por ejemplo, utilizando `HTML2Text` o `Beautiful Soup`).


In [1]:
!pip install --upgrade pip
!pip install -q typing-inspect==0.8.0 typing_extensions==4.5.0
!pip install pydantic==1.10.13
!pip install -q tiktoken cohere fastapi kaleido python-multipart uvicorn
!pip install -q openai langchain playwright beautifulsoup4 html2text
!playwright install

# Set env var OPENAI_API_KEY or load from a .env file:
# import dotenv
# dotenv.load_dotenv()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydantic-core 2.14.5 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 9.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.11
    Uninstalling pydantic-1.10.11:
      Successfully uninstalled pydantic-1.10.11
  error: subprocess-exited-with-error
  
  × Building wheel for fastavro (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [67 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-11.1-arm64-cpython-39
      creating build/lib.macosx-11.1-arm64-cpython-39/f

Raspando contenido HTML utilizando una instancia *headless* de Chromium.

* La naturaleza asíncrona del proceso de raspado se maneja utilizando la biblioteca asyncio de Python.
* La interacción real con las páginas web se maneja mediante Playwright.

In [4]:
import nest_asyncio


nest_asyncio.apply()

In [43]:
from langchain.document_loaders import AsyncChromiumLoader
from langchain.document_transformers import BeautifulSoupTransformer


# Load HTML
loader = AsyncChromiumLoader(["https://www.latorre.com.gt/"])
html = loader.load()

Raspar etiquetas de contenido de texto tales como `<p>, <li>, <div>, y <a>` del contenido HTML:

* `<p>`: La etiqueta de párrafo. Define un párrafo en HTML y se utiliza para agrupar oraciones y/o frases relacionadas.

* `<li>`: La etiqueta de elemento de lista. Se utiliza dentro de listas ordenadas (`<ol>`) y no ordenadas (`<ul>`) para definir elementos individuales dentro de la lista.

* `<div>`: La etiqueta de división. Es un elemento a nivel de bloque utilizado para agrupar otros elementos en línea o a nivel de bloque.

* `<a>`: La etiqueta de anclaje. Se utiliza para definir hipervínculos.

* `<span>`: un contenedor en línea utilizado para marcar una parte de un texto o una parte de un documento.

En muchos sitios web de noticias (por ejemplo, WSJ, CNN), los titulares y resúmenes están todos en etiquetas `<span>`.

In [44]:
# Transform
bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(html, tags_to_extract=["span"])

In [45]:
from pprint import pprint


# Result
pprint(docs_transformed[0].page_content[0:500])

('Entrar 0.00 Menú Nosotros Atencion al cliente Enlaces 1. leche 2. yogurt 3. '
 'cafe 4. galletas 5. nivea 6. vino 7. pan 8. shampoo johnson 9. crema 10. '
 'helado')


In [46]:
bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(html, tags_to_extract=["a"])
pprint(docs_transformed[0])

Document(page_content='', metadata={'source': 'https://www.latorre.com.gt/'})


Estos `Documentos` ahora están preparados para su uso posterior en varias aplicaciones LLM, como se discute a continuación.

## Loader

### AsyncHtmlLoader

El [AsyncHtmlLoader](docs/integrations/document_loaders/async_html) utiliza la biblioteca `aiohttp` para realizar solicitudes HTTP asincrónicas, adecuadas para raspados más simples y ligeros.

### AsyncChromiumLoader

El [AsyncChromiumLoader](docs/integrations/document_loaders/async_chromium) utiliza Playwright para lanzar una instancia de Chromium, que puede manejar la renderización de JavaScript y interacciones web más complejas.

[Chromium](https://www.chromium.org/chromium-projects/) es uno de los navegadores compatibles con Playwright, una biblioteca utilizada para controlar la automatización del navegador.

El modo *Headless* significa que el navegador se está ejecutando sin una interfaz gráfica de usuario, lo cual es comúnmente utilizado para el raspado web.


In [51]:
from langchain.document_loaders import AsyncHtmlLoader


urls = [
    "https://www.latorre.com.gt/",
    "https://cnnespanol.cnn.com/category/centroamerica/guatemala/",
    "https://www.exito.com/"
]
loader = AsyncHtmlLoader(urls)
docs = loader.load()

Fetching pages:   0%|          | 0/3 [00:00<?, ?it/s]

Fetching pages: 100%|##########| 3/3 [00:02<00:00,  1.46it/s]


In [52]:
from langchain.document_transformers import Html2TextTransformer


html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs, tags_to_extract=["a"])

In [53]:
pprint(docs_transformed[0].page_content)

('Entrar\n'
 '\n'
 'Mis Listas\n'
 '\n'
 '0.00\n'
 '\n'
 '  * Menú\n'
 '\n'
 '  * Ofertas Publicadas\n'
 '\n'
 '  * Marcas Gold\n'
 '\n'
 '  * Essential Everyday\n'
 '\n'
 'Limpieza\n'
 '\n'
 'Bebés\n'
 '\n'
 'Mascotas\n'
 '\n'
 'Snacks\n'
 '\n'
 'Belleza\n'
 '\n'
 'Panadería\n'
 '\n'
 'Abarrotes\n'
 '\n'
 '  * Nosotros\n'
 '\n'
 '  * Blog\n'
 '\n'
 '  * Ubicaciones\n'
 '\n'
 '  * Conócenos\n'
 '\n'
 '  * Amando Guate\n'
 '\n'
 '  * Contáctanos\n'
 '\n'
 '  * Atencion al cliente\n'
 '\n'
 '  * Horario\n'
 '\n'
 '  * Preguntas Frecuentes\n'
 '\n'
 '  * Enlaces\n'
 '\n'
 '  * Términos y Condiciones\n'
 '\n'
 '  * Politicas de Privacidad\n'
 '\n'
 '#### Comunícate con nosotros\n'
 '\n'
 '###### (+502) 2376-7474\n'
 '\n'
 '###### servicioalcliente@supermercadoslatorre.com\n'
 '\n'
 '### Síguenos en nuestras redes\n'
 '\n'
 'Sitio optimizado para Chrome y FireFox\n'
 '\n'
 'Powered by\n'
 '\n')


In [57]:
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

In [58]:
pprint(docs_transformed[0].page_content)

('Entrar Mis Listas 0.00 * Menú * Ofertas Publicadas * Marcas Gold * '
 'Essential\n'
 'Everyday Limpieza Bebés Mascotas Snacks Belleza Panadería Abarrotes * '
 'Nosotros\n'
 '* Blog * Ubicaciones * Conócenos * Amando Guate * Contáctanos * Atencion al\n'
 'cliente * Horario * Preguntas Frecuentes * Enlaces * Términos y Condiciones '
 '*\n'
 'Politicas de Privacidad #### Comunícate con nosotros ###### (+502) '
 '2376-7474\n'
 '###### servicioalcliente@supermercadoslatorre.com ### Síguenos en nuestras\n'
 'redes Sitio optimizado para Chrome y FireFox Powered by\n'
 '\n')


In [54]:
pprint(docs_transformed[1].page_content)

('CNNEarrow-downclosecomment-02commentglobeplaylistsearchsocial-facebooksocial-\n'
 'googleplussocial-instagramsocial-linkedinsocial-mailsocial-moresocial-\n'
 'twittersocial-whatsapp-01social-whatsapptimestamptype-audiotype-gallery\n'
 '\n'
 'Skip to content\n'
 '\n'
 '* EE.UU.\n'
 '* Mundo\n'
 '* Israel\n'
 '* México\n'
 '* Colombia\n'
 '* Argentina\n'
 '* Latam\n'
 '* Negocios\n'
 '* Clima\n'
 '* Entretenimiento\n'
 '* Video\n'
 '* Deportes\n'
 '* Salud\n'
 '* Tecno\n'
 '\n'
 'Internacional Estados Unidos Árabe Selecciona una edición\n'
 '\n'
 '(C) 2023 Cable News Network. A Warner Bros. Discovery Company. All Rights\n'
 'Reserved.\n'
 '\n'
 'CNN Sans ™ & (C) 2023 Cable News Network.\n'
 '\n'
 'Condiciones de uso Anunciate Privacidad Contactanos Elecciones de anuncios\n'
 'Powered by WordPress VIP\n'
 '\n'
 'Internacional Estados Unidos Árabe Selecciona una edición\n'
 '\n'
 'Buscar\n'
 '\n'
 'Buscar\n'
 '\n'
 '  * Latam\n'
 '    * Argentina\n'
 '    * Colombia\n'
 '    * México\n'


In [56]:
pprint(docs_transformed[2].page_content)

('\n'
 '\n'
 "🍎📱 iPhone 15 desde 3'699.900\n"
 '\n'
 '¡Lo quiero!\n'
 '\n'
 '¿Cómo quieres recibir tu pedido?\n'
 '\n'
 '  * Mi cuenta\n'
 '\n'
 'Inicio\n'
 '\n'
 'Mercado\n'
 '\n'
 'Moda\n'
 '\n'
 'Super ofertasWhatsAppGana dinero\n'
 '\n'
 '  * \n'
 '\n'
 '##### Compra por categorías\n'
 '\n'
 '¡Super ofertas!\n'
 '\n'
 'Televisores\n'
 '\n'
 'Informática\n'
 '\n'
 'Electrodomésticos\n'
 '\n'
 'Celulares\n'
 '\n'
 'Mercado\n'
 '\n'
 'Juguetería\n'
 '\n'
 'Navidad\n'
 '\n'
 'Licores\n'
 '\n'
 'Mundo cocina\n'
 '\n'
 'Moda y accesorios\n'
 '\n'
 'Colchones\n'
 '\n'
 'Retail eCommerce Award 2023\n'
 '\n'
 'Más de 400 mil productos\n'
 '\n'
 'Compras seguras\n'
 '\n'
 'Acumulas Puntos\n'
 '\n'
 'Paga en línea o en efectivo\n'
 '\n'
 'Contáctanos\n'
 '\n'
 'Ventas telefónicas\n'
 '\n'
 '(601) 4242400\n'
 '\n'
 'Ventas Whatsapp\n'
 '\n'
 '(+57) 305 482 90 46\n'
 '\n'
 'Servicio al cliente Whatsapp\n'
 '\n'
 '(+57) 305 261 54 42\n'
 '\n'
 'Línea de servicio al cliente nacional:\n'
 '\n'
 '0

In [15]:

from langchain.document_loaders import AsyncHtmlLoader

urls = ["https://succasa.com.gt/mayoreo-envases-y-contenedores/"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  3.26it/s]


## Transformer

### HTML2Text

[HTML2Text](docs/integrations/document_transformers/html2text) proporciona una conversión directa de contenido HTML a texto plano (con formato similar a markdown) sin manipulación específica de etiquetas.

Es más adecuado para escenarios donde el objetivo es extraer texto legible por humanos sin necesidad de manipular elementos HTML específicos.

### Beautiful Soup

Beautiful Soup ofrece un control más detallado sobre el contenido HTML, permitiendo la extracción específica de etiquetas, eliminación y limpieza de contenido.

Es adecuado para casos en los que quieras extraer información específica y limpiar el contenido HTML según tus necesidades.

In [59]:
from langchain.document_loaders import AsyncHtmlLoader


urls = [
    "https://www.latorre.com.gt/",
    "https://cnnespanol.cnn.com/category/centroamerica/guatemala/",
    "https://www.exito.com/",
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&sort=num_votes,desc"
]
loader = AsyncHtmlLoader(urls)
docs = loader.load()

Fetching pages:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching pages: 100%|##########| 5/5 [00:03<00:00,  1.47it/s]


In [62]:
from langchain.document_transformers import Html2TextTransformer

html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)
pprint(docs_transformed[-1].page_content)

('Menu Movies Release CalendarTop 250 MoviesMost Popular MoviesBrowse Movies '
 'by\n'
 'GenreTop Box OfficeShowtimes & TicketsMovie NewsIndia Movie Spotlight TV '
 'Shows\n'
 "What's on TV & StreamingTop 250 TV ShowsMost Popular TV ShowsBrowse TV "
 'Shows\n'
 'by GenreTV News Watch What to WatchLatest TrailersIMDb OriginalsIMDb '
 'PicksIMDb\n'
 'Podcasts Awards & Events OscarsEmmysHoliday PicksMAMISTARmeter AwardsAwards\n'
 'CentralFestival CentralAll Events Celebs Born TodayMost Popular\n'
 'CelebsCelebrity News Community Help CenterContributor ZonePolls For '
 'Industry\n'
 'Professionals * Language English (United States) * Language * * Fully\n'
 'supported * * English (United States) * Partially supported * * Français\n'
 '(Canada) * Français (France) * Deutsch (Deutschland) * हिंदी (भारत) * '
 'Italiano\n'
 '(Italia) * Português (Brasil) * Español (España) * Español (México) All * '
 'All\n'
 '* Titles * TV Episodes * Celebs * Companies * Keywords * Advanced Search\n'
 'Watchli

In [72]:
docs_transformed

[Document(page_content='Entrar Mis Listas 0.00 * Menú * Ofertas Publicadas * Marcas Gold * Essential\nEveryday Limpieza Bebés Mascotas Snacks Belleza Panadería Abarrotes * Nosotros\n* Blog * Ubicaciones * Conócenos * Amando Guate * Contáctanos * Atencion al\ncliente * Horario * Preguntas Frecuentes * Enlaces * Términos y Condiciones *\nPoliticas de Privacidad #### Comunícate con nosotros ###### (+502) 2376-7474\n###### servicioalcliente@supermercadoslatorre.com ### Síguenos en nuestras\nredes Sitio optimizado para Chrome y FireFox Powered by\n\n', metadata={'source': 'https://www.latorre.com.gt/', 'title': 'La Torre | Supermercado Online', 'description': '¡Haz tus compras sin salir de casa! Ingresa y descubre todos los productos que tenemos para ti en latorre.com.gt. ', 'language': 'es-GT'}),
 Document(page_content='CNNEarrow-downclosecomment-02commentglobeplaylistsearchsocial-facebooksocial-\ngoogleplussocial-instagramsocial-linkedinsocial-mailsocial-moresocial-\ntwittersocial-whatsap

In [66]:
docs_transformed[0].metadata

{'source': 'https://www.latorre.com.gt/',
 'title': 'La Torre | Supermercado Online',
 'description': '¡Haz tus compras sin salir de casa! Ingresa y descubre todos los productos que tenemos para ti en latorre.com.gt. ',
 'language': 'es-GT'}

## Raspado con extracción

### LLM con llamada de funciones

El raspado web es un desafío por muchas razones.

Una de ellas es la naturaleza cambiante de los diseños y contenidos de los sitios web modernos, lo que requiere modificar los scripts de raspado para adaptarse a los cambios.

Usando *Function* (por ejemplo, OpenAI) con una cadena de extracción, evitamos tener que cambiar constantemente tu código cuando los sitios web cambian.

Estamos utilizando `gpt-3.5-turbo-0613` para garantizar el acceso a la función de *Functions* de OpenAI (aunque esto podría estar disponible para todos al momento de escribir).

También mantenemos la `temperatura` en `0` para reducir la aleatoriedad del LLM.

In [75]:
from langchain.chat_models import ChatOpenAI



llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo-0613",
    openai_api_key="PUT_YOUR_API_KEY_HERE"
)

### Definir un esquema

A continuación, defines un esquema para especificar qué tipo de datos deseas extraer.

Aquí, los nombres de las claves importan ya que le dicen al LLM qué tipo de información quieren.

Por lo tanto, sé lo más detallado posible.

En este ejemplo, queremos raspar solo el nombre y el resumen de los artículos de noticias del sitio web de [CNN en Español](https://cnnespanol.cnn.com/).

In [76]:
from langchain.chains import create_extraction_chain

schema = {
    "properties": {
        "news_article_title": {"type": "string"},
        "news_article_summary": {"type": "string"},
    },
    "required": ["news_article_title", "news_article_summary"],
}


def extract(content: str, schema: dict):
    return create_extraction_chain(schema=schema, llm=llm).run(content)

### Ejecutar el raspador web con BeautifulSoup

Como se mostró anteriormente, utilizaremos `BeautifulSoupTransformer`.

In [78]:
!pip install tiktoken

  Using cached tiktoken-0.5.1-cp39-cp39-macosx_11_0_arm64.whl.metadata (6.6 kB)
Using cached tiktoken-0.5.1-cp39-cp39-macosx_11_0_arm64.whl (925 kB)


In [79]:
import pprint

from langchain.text_splitter import RecursiveCharacterTextSplitter


def scrape_with_playwright(urls, schema):
    loader = AsyncChromiumLoader(urls)
    docs = loader.load()
    bs_transformer = BeautifulSoupTransformer()
    docs_transformed = bs_transformer.transform_documents(
        docs, tags_to_extract=["span"]
    )
    print("Extracting content with LLM")

    # Grab the first 1000 tokens of the site
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1000, chunk_overlap=0
    )
    splits = splitter.split_documents(docs_transformed)

    # Process the first split
    extracted_content = extract(schema=schema, content=splits[0].page_content)
    pprint.pprint(extracted_content)
    return extracted_content


urls = ["https://cnnespanol.cnn.com/"]
extracted_content = scrape_with_playwright(urls, schema=schema)

Extracting content with LLM
[{'news_article_summary': 'Temas del día 2 3  Minuto a minuto '
                          '(https://cnnespanol.cnn.com/category/minuto-a-minuto/)   '
                          'Noticias de EE.UU. '
                          '(https://cnnespanol.cnn.com/category/estados-unidos/)   '
                          'Empresas '
                          '(https://cnnespanol.cnn.com/category/empresas/)   '
                          'Argentina '
                          '(https://cnnespanol.cnn.com/category/cono-sur/argentina/)   '
                          'Historias Humanas '
                          '(https://cnnespanol.cnn.com/category/historias-humanas-2/)   '
                          'Francia '
                          '(https://cnnespanol.cnn.com/category/francia/)   '
                          'Argentina '
                          '(https://cnnespanol.cnn.com/category/cono-sur/argentina/)   '
                          'Noticias de EE.UU. '
                

In [93]:
# PUT YOUR CODE HERE